# Topic Extraction

[Topic Modeling with Gensim](https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/)

In [ ]:
import pandas as pd

# Read cleaned data
df = pd.read_csv('https://raw.githubusercontent.com/nphan20181/nlp_project/main/data/data_cleaned.csv')

# Preview cleaned data
df.head()

In [ ]:
df = df[['Text_Cleaned', 'Title']]
df.head()

In [15]:
# Import Gensim libraries
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# Import plotting tools
!pip install pyLDAvis==2.1.2
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

from pprint import pprint

In [ ]:
data = df.Text_Cleaned.values.tolist()

pprint(data[:1])

In [19]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

print(data_words[:1])

[['electronic', 'health', 'record', 'ehr', 'set', 'clinical', 'health', 'information', 'useful', 'patient', 'treatment', 'eg', 'clinical', 'laboratory', 'report', 'discharge', 'letter', 'emergency', 'report', 'produce', 'hospital', 'patient', 'summary', 'ps', 'produce', 'general', 'practitioner', 'gp', 'datum', 'two', 'different', 'state', 'validate', 'eg', 'document', 'digitally', 'sign', 'doctor', 'validate', 'eg', 'health', 'datum', 'pressure', 'record', 'patient', 'autonomously', 'scenario', 'typically', 'talk', 'personal', 'health', 'record', 'phr', 'contexts', 'interest', 'point', 'view', 'patient', 'often', 'refer', 'concept', 'relate', 'health', 'datum', 'document', 'alessandra', 'pieroni', 'et', 'al', 'associate', 'to', 'whole', 'hospitalization', 'ward', 'also', 'consider', 'hospitalization', 'outpatient', 'episode', 'within', 'ward', 'itself', 'gp', 'whole', 'hospital', 'consider', 'episode', 'different', 'ward', 'hospitalization', 'too', 'health', 'datum', 'document', 'pati

In [21]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100)
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)

# Faster way to get a sentecne clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

/usr/local/lib/python3.7/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


['electronic_health_record_ehr', 'set', 'clinical', 'health', 'information', 'useful', 'patient', 'treatment', 'eg', 'clinical', 'laboratory', 'report', 'discharge', 'letter', 'emergency', 'report', 'produce', 'hospital', 'patient', 'summary', 'ps', 'produce', 'general', 'practitioner', 'gp', 'datum', 'two', 'different', 'state', 'validate', 'eg', 'document', 'digitally_sign', 'doctor', 'validate', 'eg', 'health', 'datum', 'pressure', 'record', 'patient', 'autonomously', 'scenario', 'typically', 'talk', 'personal', 'health', 'record', 'phr', 'contexts', 'interest', 'point', 'view', 'patient', 'often', 'refer', 'concept', 'relate', 'health', 'datum', 'document', 'alessandra_pieroni', 'et', 'al', 'associate', 'to', 'whole', 'hospitalization', 'ward', 'also', 'consider', 'hospitalization', 'outpatient', 'episode', 'within', 'ward', 'itself', 'gp', 'whole', 'hospital', 'consider', 'episode', 'different', 'ward', 'hospitalization', 'too', 'health', 'datum', 'document', 'patient', 'regardles

In [22]:
def make_bigrams(texts):
  return [bigram_mod[doc] for doc in texts]

def make_trigrams(tests):
  return [trigram_mod[bigram_mod[doc]] for doc in texts] 

In [24]:
data_words_bigrams = make_bigrams(data_words)

print(data_words_bigrams[:1])

[['electronic_health', 'record_ehr', 'set', 'clinical', 'health', 'information', 'useful', 'patient', 'treatment', 'eg', 'clinical', 'laboratory', 'report', 'discharge', 'letter', 'emergency', 'report', 'produce', 'hospital', 'patient', 'summary', 'ps', 'produce', 'general', 'practitioner', 'gp', 'datum', 'two', 'different', 'state', 'validate', 'eg', 'document', 'digitally_sign', 'doctor', 'validate', 'eg', 'health', 'datum', 'pressure', 'record', 'patient', 'autonomously', 'scenario', 'typically', 'talk', 'personal', 'health', 'record', 'phr', 'contexts', 'interest', 'point', 'view', 'patient', 'often', 'refer', 'concept', 'relate', 'health', 'datum', 'document', 'alessandra_pieroni', 'et', 'al', 'associate', 'to', 'whole', 'hospitalization', 'ward', 'also', 'consider', 'hospitalization', 'outpatient', 'episode', 'within', 'ward', 'itself', 'gp', 'whole', 'hospital', 'consider', 'episode', 'different', 'ward', 'hospitalization', 'too', 'health', 'datum', 'document', 'patient', 'regar

In [27]:
# Create Dictionary
id2word = corpora.Dictionary(data_words_bigrams)

# Create Corpus
texts = data_words_bigrams

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

print(corpus[:1])

[[(0, 3), (1, 3), (2, 1), (3, 1), (4, 1), (5, 3), (6, 6), (7, 1), (8, 19), (9, 1), (10, 1), (11, 2), (12, 1), (13, 3), (14, 1), (15, 2), (16, 1), (17, 1), (18, 2), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 2), (27, 3), (28, 4), (29, 3), (30, 37), (31, 13), (32, 1), (33, 15), (34, 1), (35, 1), (36, 4), (37, 15), (38, 2), (39, 2), (40, 3), (41, 1), (42, 5), (43, 2), (44, 1), (45, 1), (46, 1), (47, 6), (48, 1), (49, 1), (50, 18), (51, 1), (52, 2), (53, 1), (54, 1), (55, 1), (56, 1), (57, 1), (58, 1), (59, 1), (60, 2), (61, 4), (62, 2), (63, 3), (64, 1), (65, 2), (66, 10), (67, 1), (68, 1), (69, 2), (70, 1), (71, 1), (72, 1), (73, 1), (74, 2), (75, 22), (76, 2), (77, 1), (78, 2), (79, 1), (80, 2), (81, 1), (82, 1), (83, 6), (84, 5), (85, 2), (86, 1), (87, 1), (88, 1), (89, 2), (90, 3), (91, 1), (92, 4), (93, 7), (94, 4), (95, 1), (96, 4), (97, 1), (98, 2), (99, 3), (100, 1), (101, 1), (102, 1), (103, 1), (104, 1), (105, 1), (106, 1), (107, 1), (108, 1), (109, 1), 

In [ ]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(
    corpus = corpus,
    id2word = id2word,
    num_topics = 50,
    random_state = 100,
    update_every=1,
    chunksize=100,
    passes = 10,
    alpha = 'auto',
    per_word_topics=True
)

In [ ]:
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

In [30]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
30     0.241312 -0.005029       1        1  9.028580
0      0.236267  0.088031       2        1  8.494822
23     0.238618 -0.061237       3        1  6.465302
45     0.219723  0.009429       4        1  5.302444
48     0.201519 -0.147493       5        1  5.192743
29     0.213928 -0.012879       6        1  4.644257
9      0.229159 -0.079782       7        1  4.197208
6      0.214980  0.174212       8        1  3.986635
32     0.212757 -0.003934       9        1  3.795770
11     0.206198 -0.067248      10        1  3.740200
39    -0.210435 -0.012460      11        1  3.060388
34     0.181767  0.116356      12        1  2.785151
2      0.154911  0.101088      13        1  2.526788
20     0.172227 -0.053338      14        1  2.505858
21     0.129898  0.079725      15        1  2.405759
17    -0.225229 -0.010459      16        1  2.275554
35     0.124166  0.058001      17        1  2.262324
12     0.104554  0.204599      18        1  2.256771
43     0.180505  0.055763      19        1  2.221118
41     0.156510 -0.216221      20        1  2.064715
46     0.115660 -0.092682      21        1  1.872878
22     0.105229  0.028014      22        1  1.824878
31     0.133989 -0.128569      23        1  1.808877
10     0.042577 -0.149184      24        1  1.549579
16     0.103065  0.197558      25        1  1.470939
1      0.089041  0.107953      26        1  1.310302
19     0.082061 -0.134396      27        1  1.283617
28     0.070986  0.086850      28        1  1.115544
33    -0.032018 -0.109423      29        1  0.855750
5     -0.051954  0.011660      30        1  0.696722
14    -0.080289 -0.117031      31        1  0.625416
13    -0.069743  0.000600      32        1  0.598947
15    -0.087033 -0.058646      33        1  0.585166
26    -0.043955 -0.108444      34        1  0.570391
24    -0.136433  0.087226      35        1  0.501772
36    -0.184745  0.051287      36        1  0.426880
7     -0.188463 -0.006289      37        1  0.426145
3     -0.148207  0.076698      38        1  0.378932
40    -0.227180  0.003340      39        1  0.340939
49    -0.197002  0.019380      40        1  0.302024
42    -0.177787 -0.015240      41        1  0.298586
38    -0.223860  0.006494      42        1  0.286984
44    -0.192874 -0.027813      43        1  0.258662
4     -0.224795  0.027749      44        1  0.250935
25    -0.200221 -0.007721      45        1  0.245166
8     -0.243716  0.005001      46        1  0.223949
47    -0.246032  0.007506      47        1  0.204323
37    -0.252749  0.003164      48        1  0.182203
18    -0.255507  0.007140      49        1  0.153818
27    -0.261379  0.010692      50        1  0.137293, topic_info=                     Term          Freq  ...  logprob  loglift
4412                   ˇˇ  14985.000000  ...  30.0000  30.0000
469                 image  17578.000000  ...  29.0000  29.0000
4170                   ˆˆ  12354.000000  ...  28.0000  28.0000
645                  node  16252.000000  ...  27.0000  27.0000
610                 model  16149.000000  ...  26.0000  26.0000
...                   ...           ...  ...      ...      ...
21798  gamma_distribution     20.866432  ...  -5.5114   6.3428
11512             poisson     33.414987  ...  -5.0405   5.9028
7808                   dn     35.013421  ...  -4.9938   5.3088
3155            retrieval     33.765425  ...  -5.0301   2.6600
378               formula     23.963305  ...  -5.3730   3.1379

[2952 rows x 6 columns], token_table=       Topic      Freq          Term
term                                
5409       1  0.138448            ab
5409       7  0.026193            ab
5409      11  0.263799            ab
5409      14  0.084191            ab
5409      16  0.265670            ab
...      ...       ...           ...
4576      11  0.999397         ˇˇˇˇˇ
4456      11  0.995792        ˇˇˇˇˇˇ
64437     47  0.976874  ﬁapplication
